Table RAS45003 Reported road casualties by severity (estimates): Great Britain, quarterly and annual

In [1]:
from gssutils import *

scraper = Scraper('https://www.gov.uk/government/statistical-data-sets/ras45-quarterly-statistics')
scraper

## Quarterly statistics on reported road accidents (RAS45)

Data on reported road accidents, produced by Department for Transport.

### Distributions

1. Reported road casualties by severity (estimates): Great Britain, rolling annual totals, quarterly ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754497/ras45001.ods))
1. Reported road casualties by severity and road user type (estimates): Great Britain, rolling annual totals, updated quarterly ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754502/ras45006.ods))
1. Reported road accidents by severity (estimates): Great Britain, rolling annual totals, updated quarterly ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754500/ras45004.ods))
1. Reported road accidents, by road type (estimates): Great Britain, rolling annual totals, updated quarterly ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/588103/ras45009.ods))
1. Reported road casualties by severity (estimates): Great Britain, latest available quarter ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754498/ras45002.ods))
1. Reported road casualties by severity and road user type (estimates): Great Britain, latest available quarter ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754503/ras45007.ods))
1. Reported road accidents by severity (estimates): Great Britain, latest available quarter ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754501/ras45005.ods))
1. Reported road accidents by road type (estimates): Great Britain, latest available quarter ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/588104/ras45010.ods))
1. Reported road casualties by severity (estimates): Great Britain, quarterly and annual ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754499/ras45003.ods))
1. Reported road casualties by road user type (estimates): Great Britain, quarterly and annual ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754504/ras45008.ods))
1. Reported road casualties by police force area, rolling annual totals, updated quarterly ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/754505/ras45011.ods))


In [2]:
df = scraper.distribution(
    title='Reported road casualties by severity (estimates): Great Britain, quarterly and annual'
).as_pandas(sheet_name='ras45003', start_row = 6,
            row_limit = 30, start_column = 0, column_limit = 7)
observations = df.rename(columns=df.iloc[0]).drop(df.index[0])
observations

,Year and quarter,,Killed,Killed or seriously injured,Slightly injured,All casualties,Motor traffic 1 (billion vehicle miles)
1,2010-2014 average,,1798.6,24455.6,172838,197294,304.99
2,2008,,2538,28572,202333,230905,311.04
3,2009,,2222,26912,195234,222146,308.09
4,2010,,1850,24510,184138,208648,303.19
5,2011,,1901,25023,178927,203950,303.78
6,2012,,1754,24793,170930,195723,302.64
7,2013,,1713,23370,160300,183670,303.7
8,2014,,1775,24582,169895,194477,311.62
9,2015,,1730,23874,162315,186189,316.72
10,2016,,1792,25893,155491,181384,323.66


In [3]:
observations.columns.values[0] = 'Year'
observations.columns.values[1] = 'Quarter'
observations.columns.values[2] = 'killed'
observations.columns.values[3] = 'ksi'
observations.columns.values[4] = 'slightly-injured'
observations.columns.values[5] = 'total'
observations.columns.values[6] = 'motor-traffic'

In [4]:
new_table = pd.melt(observations,
                       ['Year','Quarter'], var_name="Severity",
                       value_name="Value")

In [5]:
def user_perc(x):
    
    if str(x) == '2010-2014 average':
        return 'Average Count'
    else:
        return 'Count'
    
new_table['Measure Type'] = new_table.apply(lambda row: user_perc(row['Year']), axis = 1)

In [6]:
def user_perc(x):
    
    if (str(x) == 'Q1(P)') | (str(x) == 'Q2(P)') :
        return 'Provisional Estimate'
    else:
        return 'Original Value'
    
new_table['Revision'] = new_table.apply(lambda row: user_perc(row['Quarter']), axis = 1)

In [7]:
new_table.Year = new_table.Year[new_table.Year.str.strip() != '']

In [8]:
new_table.Year = new_table.Year.ffill()

In [9]:
new_table['Quarter'] = new_table['Quarter'].str.rstrip('(P)')

In [10]:
new_table['Year'] = new_table['Year'].map(str)
new_table['Quarter'] = new_table['Quarter'].map(str)
def user_perc(x,y):    
    if x.strip() == '':
        return 'year/'+ y
    else:
        return 'quarter/'+ y +'-'+ x        
    
new_table['Period'] = new_table.apply(lambda row: user_perc(row['Quarter'], row['Year']), axis = 1)

In [11]:
new_table['Unit'] = 'Casualities'

In [12]:
new_table['Period'] = new_table['Period'].map(
    lambda x: {
        'year/2010-2014 average' : 'gregorian-interval/2010-01-01T00:00:00/P4Y'
        }.get(x, x))

In [13]:
new_table = new_table[['Period','Severity','Measure Type','Value','Unit','Revision']]

In [14]:
from pathlib import Path

destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('RAS45003.csv'), index = False)

In [15]:
new_table

,Period,Severity,Measure Type,Value,Unit,Revision
0,gregorian-interval/2010-01-01T00:00:00/P4Y,killed,Average Count,1798.6,Casualities,Original Value
1,year/2008,killed,Count,2538,Casualities,Original Value
2,year/2009,killed,Count,2222,Casualities,Original Value
3,year/2010,killed,Count,1850,Casualities,Original Value
4,year/2011,killed,Count,1901,Casualities,Original Value
5,year/2012,killed,Count,1754,Casualities,Original Value
6,year/2013,killed,Count,1713,Casualities,Original Value
7,year/2014,killed,Count,1775,Casualities,Original Value
8,year/2015,killed,Count,1730,Casualities,Original Value
9,year/2016,killed,Count,1792,Casualities,Original Value
